In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import psycopg2
from dotenv import load_dotenv
import os
import joblib
import matplotlib.pyplot as plt

In [2]:

model = joblib.load('modelo/modelo.pkl')

print(model)

c:\Users\gutil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\gutil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.3.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\gutil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimat

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=[5],
                               random_state=352))])


In [3]:

Xnew = [[120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000]]
ypred = model.predict(Xnew)  
print(ypred)

[0.]


In [7]:
env_path="C:\\Users\\gutil\\OneDrive - Universidad de los andes\\Escritorio\\Ingenieria Industrial\\2024-01\\Analitica Computacional Para la Toma de Decisiones\\Clases\\Clase 11\\actd_clase21\\actd_clase21\\env\\app.env"
# load env 
load_dotenv(dotenv_path=env_path)
# extract env variables
USER=os.getenv('USER')
PASSWORD=os.getenv('PASSWORD')
HOST=os.getenv('HOST')
PORT=os.getenv('PORT')
DBNAME=os.getenv('DBNAME')

#connect to DB
print(DBNAME)
print(USER)
print(PASSWORD)
print(HOST)
print(PORT)


world
postgres
actd2024
actd24.c2ejf18x6i7a.us-east-1.rds.amazonaws.com
5432
